# Factor analysis


## TODO

1. Run regression with X being categorical variable for quintiles (logistic regression?)
2. p37 : explore the cryptocurrency competition effect on asset valuation (price)
3. Extend data period to 2023 (before/after covid?)
4. Add macroeconomic data and run together (construct the variables you can construct)
   - VIX - measure of market volatility
   - Trade-Weighted U.S. Dollar Index provided by the Federal Reserve
   - Add GPU cost and chip price

### How to do regressions

Each week, we sort the returns of individual cryptocurrencies into quintile portfolios based on the value of a given characteristic. We track the return of each portfolio in the week that follows and calculate each portfolio’s average excess return over the risk-free rate. We then form a long-short strategy based on the difference between the fifth and the first quintiles.


**It's not stated whether they use CUMULATIVE OR COMPOUNDING RETURNS**


#### Factors list

`Size`

1. MCAP - log last-day market capitalization in the portfolio formation week
2. PRC - log last-day price in the portfolio formation week
3. MAXDPRC - Maximum price of the portfolio formation week 

`Momentum`

4. r 1,0 - past one-week return
5. r 2,0 - past two-week return
6. r 3,0 - past three-week return
7. r 4,0 - past four-week return
8. r 4,1 - past one-to-four-week return

`Volume`

9. PRCVOL - log avg daily volume times price in the portfolio formation week

`Volatility`

10. STDPRCVOL - log standard deviation of price volume in the portfolio formation week




In [1]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
# from scipy.stats.mstats import winsorize
import numpy as np

In [2]:
db = pd.read_csv('./inter/processed_data.csv')
db['date'] = pd.to_datetime(db['date'])
db.tail()

,date,coin_id,market_cap,close,volume_24h,symbol,status,category,year,dy,week,close_t0,daily_ret,weekly_ret,one_mth_treasury
840738,2020-07-27,5864,8.552109e+07,2876.587797,3.992772e+06,YFI,active,token,2020,209,30,3099.858747,-0.072026,1.391436,0.10
840739,2020-07-28,5864,1.041069e+08,3486.831260,7.643382e+06,YFI,active,token,2020,210,30,2876.587797,0.212141,1.391436,0.09
840740,2020-07-29,5864,1.344364e+08,4497.657841,1.794091e+07,YFI,active,token,2020,211,31,3486.831260,0.289898,0.252341,0.09
840741,2020-07-30,5864,1.302110e+08,4353.301602,4.471048e+06,YFI,active,token,2020,212,31,4497.657841,-0.032096,0.252341,0.10
840742,2020-07-31,5864,1.295505e+08,4329.525667,1.204022e+07,YFI,active,token,2020,213,31,4353.301602,-0.005462,0.252341,0.09


In [3]:
# Remove the last three obs for 2020 because 3 days can't be considered 1 week
db = db.iloc[:-3, :]
db.tail()

# ---> Data till 2020-07-28

,date,coin_id,market_cap,close,volume_24h,symbol,status,category,year,dy,week,close_t0,daily_ret,weekly_ret,one_mth_treasury
840735,2020-07-24,5864,7.694450e+07,3239.783482,5.252213e+05,YFI,active,token,2020,206,30,1894.562951,0.710043,1.391436,0.10
840736,2020-07-25,5864,9.468365e+07,3453.795356,2.253002e+06,YFI,active,token,2020,207,30,3239.783482,0.066057,1.391436,0.00
840737,2020-07-26,5864,9.096854e+07,3099.858747,1.230522e+06,YFI,active,token,2020,208,30,3453.795356,-0.102478,1.391436,0.00
840738,2020-07-27,5864,8.552109e+07,2876.587797,3.992772e+06,YFI,active,token,2020,209,30,3099.858747,-0.072026,1.391436,0.10
840739,2020-07-28,5864,1.041069e+08,3486.831260,7.643382e+06,YFI,active,token,2020,210,30,2876.587797,0.212141,1.391436,0.09


### Create weekly factors 


In [4]:
# Calculating MCAP

db_MCAP = db.groupby(['coin_id', 'year', 'week'])['market_cap'].last().reset_index(name = 'MCAP')
db_MCAP['MCAP'] = np.log(db_MCAP['MCAP'])
db_MCAP.head(2)


,coin_id,year,week,MCAP
0,1,2014,1,23.006495
1,1,2014,2,23.047469


In [5]:
# Calculating PRC

db_PRC = db.groupby(['coin_id', 'year', 'week'])['close'].last().reset_index(name = 'PRC')
db_PRC['PRC'] = np.log(db_PRC['PRC'])


In [6]:
# Calculating MAXDPRC

db_MAXDPRC = db.groupby(['coin_id', 'year', 'week'])['close'].max().reset_index(name = 'MAXDPRC')


In [7]:
# Calculating r1

db = db.sort_values(by=['coin_id', 'year', 'week'])
db_r1 = db.groupby(['coin_id', 'year', 'week'])['daily_ret'].sum().reset_index(name = 'weekly_ret')
db_r1['r1'] = db_r1['weekly_ret'].shift(1)
db_r1 = db_r1.drop('weekly_ret', axis=1)


In [8]:
# Calculating r2

db = db.sort_values(by=['coin_id', 'year', 'week'])
db_r2 = db.groupby(['coin_id', 'year', 'week'])['daily_ret'].sum().reset_index(name = 'weekly_ret')
db_r2['l1'] = db_r2['weekly_ret'].shift(1)
db_r2['l2'] = db_r2['weekly_ret'].shift(2)

db_r2['r2'] = db_r2['l1'] + db_r2['l2']

db_r2 = db_r2.drop(['weekly_ret', 'l1', 'l2'], axis=1)

In [9]:
# Calculating r3

db = db.sort_values(by=['coin_id', 'year', 'week'])
db_r3 = db.groupby(['coin_id', 'year', 'week'])['daily_ret'].sum().reset_index(name = 'weekly_ret')
db_r3['l1'] = db_r3['weekly_ret'].shift(1)
db_r3['l2'] = db_r3['weekly_ret'].shift(2)
db_r3['l3'] = db_r3['weekly_ret'].shift(3)

db_r3['r3'] = db_r3['l1'] + db_r3['l2'] + db_r3['l3']

db_r3.head(10)
db_r3 = db_r3.drop(['weekly_ret', 'l1', 'l2',  'l3'], axis=1)

In [10]:
# Calculating r4

db = db.sort_values(by=['coin_id', 'year', 'week'])
db_r4 = db.groupby(['coin_id', 'year', 'week'])['daily_ret'].sum().reset_index(name = 'weekly_ret')
db_r4['l1'] = db_r4['weekly_ret'].shift(1)
db_r4['l2'] = db_r4['weekly_ret'].shift(2)
db_r4['l3'] = db_r4['weekly_ret'].shift(3)
db_r4['l4'] = db_r4['weekly_ret'].shift(4)

db_r4['r4'] = db_r4['l1'] + db_r4['l2'] + db_r4['l3'] + db_r4['l4']

db_r4 = db_r4.drop(['weekly_ret', 'l1', 'l2', 'l3', 'l4'], axis=1)

In [11]:
# Calculating r4_1

db = db.sort_values(by=['coin_id', 'year', 'week'])
db_r4_1 = db.groupby(['coin_id', 'year', 'week'])['daily_ret'].sum().reset_index(name = 'weekly_ret')
db_r4_1['l1'] = db_r4_1['weekly_ret'].shift(1)
db_r4_1['l4'] = db_r4_1['weekly_ret'].shift(4)

db_r4_1['r4_1'] = db_r4_1['l1'] - db_r4_1['l4']

db_r4_1 = db_r4_1.drop(['weekly_ret', 'l1', 'l4'], axis=1)

In [12]:
# Calculating weekly PRCVOL

db['volxclose'] = db['volume_24h'] * db['close']
db_PRCVOL = db.groupby(['coin_id', 'year', 'week'])['volxclose'].mean().reset_index(name = 'PRCVOL')
db_PRCVOL['PRCVOL'] = np.log(db_PRCVOL['PRCVOL'])


In [13]:
# Calculating weekly STDPRCVOL

db_STDPRCVOL = db.groupby(['coin_id', 'year', 'week'])['volxclose'].std().reset_index(name = 'STDPRCVOL')

db_STDPRCVOL['STDPRCVOL'] = np.log(db_STDPRCVOL['STDPRCVOL'])

# db_STDPRCVOL = db_STDPRCVOL[db_STDPRCVOL['STDPRCVOL'] != -np.inf]
db_STDPRCVOL['STDPRCVOL'] = db_STDPRCVOL['STDPRCVOL'].replace(-np.inf, 0)


/home/lada/Desktop/UniCourses/Semester1_FinancialDataAnalysis/FinalTask/CryptoProject/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
db_STDPRCVOL[db_STDPRCVOL['STDPRCVOL'] == 0]


,coin_id,year,week,STDPRCVOL
19014,594,2019,11,0.0
27005,1106,2019,41,0.0
56240,1967,2019,10,0.0
75762,2382,2020,28,0.0
88582,2599,2020,9,0.0
91419,2657,2019,22,0.0
98516,2869,2020,29,0.0
100037,2912,2020,28,0.0
107122,3266,2018,39,0.0
107123,3266,2018,40,0.0


### Create weekly treasuries var


In [15]:
db_dtreasury = pd.read_csv("./input/daily_treasury_bill_rates_2011-2020.csv")
db_dtreasury['date'] = pd.to_datetime(db_dtreasury['date'])


/tmp/ipykernel_20238/1316669408.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  db_dtreasury['date'] = pd.to_datetime(db_dtreasury['date'])


In [16]:
db_dtreasury['year'] = db_dtreasury['date'].dt.year
db_dtreasury['dy'] = db_dtreasury['date'].dt.dayofyear
db_dtreasury = db_dtreasury.sort_values(by='date')

In [17]:
def define_week(day):
    if day % 7 == 0:
        return day // 7
    elif day in [365, 366]:
        return day // 7
    else:
        return day // 7 + 1

db_dtreasury['week'] = db_dtreasury['dy'].apply(define_week)


In [18]:
# db_dtreasury = db_dtreasury.drop('dy', axis=1)
# db_dtreasury = db_dtreasury.drop('date', axis=1)
db_wtreasury = db_dtreasury.groupby(['year', 'week'])['one_mth_treasury'].mean().reset_index(name='tres')
db_wtreasury = db_wtreasury.drop_duplicates(subset=['year', 'week'], keep='first')

db_wtreasury['yyyyww'] = db_wtreasury['year'].astype(str) + db_wtreasury['week'].astype(str)
db_wtreasury['yyyyww'] = db_wtreasury['yyyyww'].astype(int)

db_wtreasury = db_wtreasury.drop('year', axis=1)
db_wtreasury = db_wtreasury.drop('week', axis=1)

db_wtreasury['tres'] = db_wtreasury['tres'] / 4
db_wtreasury['tres'] = db_wtreasury['tres'] / 100


In [19]:
db_wtreasury

,tres,yyyyww
0,0.000025,20141
1,0.000010,20142
2,0.000025,20143
3,0.000070,20144
4,0.000135,20145
...,...,...
359,0.000194,202048
360,0.000195,202049
361,0.000180,202050
362,0.000190,202051


In [20]:
db_wtreasury.to_csv("./inter/wtreasury.csv", index=False)

### Megring all factors into one dataset


In [21]:
from functools import reduce

In [22]:
db_list = [db_MCAP, db_PRC, db_MAXDPRC,
           db_r1, db_r2, db_r3,
           db_r4, db_r4_1, db_PRCVOL, db_STDPRCVOL]

db_factors = reduce(lambda left, right: pd.merge(left, right, on=['coin_id', 'year', 'week'], how='left'), db_list)


In [23]:
db_factors.head(7)

,coin_id,year,week,MCAP,PRC,MAXDPRC,r1,r2,r3,r4,r4_1,PRCVOL,STDPRCVOL
0,1,2014,1,23.006495,6.687109,953.289978,NaN,NaN,NaN,NaN,NaN,24.566359,23.917686
1,1,2014,2,23.047469,6.725358,913.950012,0.058930,NaN,NaN,NaN,NaN,24.382460,23.418371
2,1,2014,3,23.086031,6.761469,870.960022,0.043129,0.102059,NaN,NaN,NaN,23.780342,22.514618
3,1,2014,4,23.027175,6.700128,885.280029,0.038573,0.081702,0.140632,NaN,NaN,23.970173,23.001259
4,1,2014,5,23.048526,6.718965,832.580017,-0.046738,-0.008165,0.034964,0.093894,-0.105668,23.426512,22.294408
5,1,2014,6,22.842387,6.510511,811.909973,0.019153,-0.027585,0.010989,0.054118,-0.023975,24.536094,23.943215
6,1,2014,7,22.774698,6.440308,661.989990,-0.201694,-0.182540,-0.229279,-0.190705,-0.240267,23.983473,23.662050


In [24]:
db_weekly_mcap = db.groupby(['coin_id', 'year', 'week'])['market_cap'].last().reset_index(name='MCAP_nolog')

# db_weekly_mcap['lag1MCAP'] = db_weekly_mcap['MCAP'].shift(1)

# db_weekly_mcap = db_weekly_mcap.drop('MCAP', axis=1)

In [25]:
db_fin = pd.merge(db_factors, db_weekly_mcap, on=['coin_id', 'year', 'week'], how='left')


In [26]:
db_fin.head()

,coin_id,year,week,MCAP,PRC,MAXDPRC,r1,r2,r3,r4,r4_1,PRCVOL,STDPRCVOL,MCAP_nolog
0,1,2014,1,23.006495,6.687109,953.289978,NaN,NaN,NaN,NaN,NaN,24.566359,23.917686,9.808300e+09
1,1,2014,2,23.047469,6.725358,913.950012,0.058930,NaN,NaN,NaN,NaN,24.382460,23.418371,1.021854e+10
2,1,2014,3,23.086031,6.761469,870.960022,0.043129,0.102059,NaN,NaN,NaN,23.780342,22.514618,1.062028e+10
3,1,2014,4,23.027175,6.700128,885.280029,0.038573,0.081702,0.140632,NaN,NaN,23.970173,23.001259,1.001325e+10
4,1,2014,5,23.048526,6.718965,832.580017,-0.046738,-0.008165,0.034964,0.093894,-0.105668,23.426512,22.294408,1.022934e+10


In [27]:
len(db_fin[db_fin['STDPRCVOL'] == np.nan])

0

In [28]:
db_weekly_ret = db.groupby(['coin_id', 'year', 'week'])['daily_ret'].sum().reset_index(name='weekly_ret')

db_weekly_ret['next_week_ret'] = db_weekly_ret['weekly_ret'].shift(-1)


In [29]:
db_fin = db_fin.merge(db_weekly_ret, on=['coin_id', 'year', 'week'], how='left')


In [30]:
db_fin['MCAP_nolog'] = db_fin['MCAP_nolog'] / 100000


In [31]:
db_fin.head()

,coin_id,year,week,MCAP,PRC,MAXDPRC,r1,r2,r3,r4,r4_1,PRCVOL,STDPRCVOL,MCAP_nolog,weekly_ret,next_week_ret
0,1,2014,1,23.006495,6.687109,953.289978,NaN,NaN,NaN,NaN,NaN,24.566359,23.917686,98082.996000,0.058930,0.043129
1,1,2014,2,23.047469,6.725358,913.950012,0.058930,NaN,NaN,NaN,NaN,24.382460,23.418371,102185.360718,0.043129,0.038573
2,1,2014,3,23.086031,6.761469,870.960022,0.043129,0.102059,NaN,NaN,NaN,23.780342,22.514618,106202.828751,0.038573,-0.046738
3,1,2014,4,23.027175,6.700128,885.280029,0.038573,0.081702,0.140632,NaN,NaN,23.970173,23.001259,100132.514839,-0.046738,0.019153
4,1,2014,5,23.048526,6.718965,832.580017,-0.046738,-0.008165,0.034964,0.093894,-0.105668,23.426512,22.294408,102293.425765,0.019153,-0.201694


In [ ]:
db_fin = db_fin.fillna(0)

db_fin.to_csv("./inter/final_factor_data.csv", index=False)

## Simple regressions



In [ ]:
# import statsmodels.api as sm
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from sklearn.metrics import mean_squared_error


In [ ]:
var_list = ['MCAP', 'PRC', 'MAXDPRC', 'r1', 'r2', 'r3', 'r4', 'r4_1', 'PRCVOL', 'STDPRCVOL']

In [ ]:
def regression_tables(db_fin, var_list, verbose=0, summary_pics=0):
    for var in var_list:
        db_var = db_fin[[
            'coin_id', 'year', 'week', var, 'MCAP_nolog', 'weekly_ret', 'next_week_ret'
        ]]

        db_var = db_var.sort_values(by=['year', 'week', var], ascending = True)
        db_var['qt'] = pd.qcut(db_var[var], q=5, labels=[1, 2, 3, 4, 5])
        db_var = db_var.sort_values(by=['year', 'week', 'qt'], ascending = True)
        suml1MCAP = db_var.groupby(['year', 'week', 'qt'])['MCAP_nolog'].sum().reset_index(name='suml1MCAP')
        db_var = db_var.merge(suml1MCAP, on=['year', 'week', 'qt'], how='left')
        db_var['lead1wret'] = db_var['next_week_ret'] * db_var['MCAP_nolog'] / db_var['suml1MCAP']
        qt_lead1wret = db_var.groupby(['year', 'week', 'qt'])['lead1wret'].sum().reset_index(name='lead1wret')

        # Create 'yyyyww var'
        qt_lead1wret['yyyyww'] = qt_lead1wret['year'].astype(str) + qt_lead1wret['week'].astype(str)
        qt_lead1wret['yyyyww'] = qt_lead1wret['yyyyww'].astype(int)
        
        lead1wret_pivot = qt_lead1wret.pivot(index='yyyyww', columns='qt', values='lead1wret')
        lead1wret_pivot['5_1'] = lead1wret_pivot.iloc[:, 4] - lead1wret_pivot.iloc[:, 0]
        lead1wret_pivot['int'] = 1
        
        # ---------------------------------------------------------------------------------------------------------
        # Remove rows with 0
        lead1wret_pivot = lead1wret_pivot.merge(db_wtreasury, on='yyyyww', how='left')
        
        new_column_names = {
            'yyyyww': 'yyyyww', 
            1: 'qt_1',
            2: 'qt_2',
            3: 'qt_3',
            4: 'qt_4',
            5: 'qt_5',
            '5_1': 'qt_5_1',
            'int': 'int',
            'tres': 'tres'
        }
        lead1wret_pivot = lead1wret_pivot.rename(columns=new_column_names)        
        for col_name in lead1wret_pivot.columns:
            if col_name not in ['yyyyww', 'int', 'tres']:
                lead1wret_pivot[col_name] = lead1wret_pivot[col_name] - lead1wret_pivot['tres']
        
        lead1wret_pivot = lead1wret_pivot.drop('tres', axis=1)
        lead1wret_pivot = lead1wret_pivot.drop('yyyyww', axis=1)
        # ------------------------------------------------------------------------------------------------------------------------------------

        db_reg_sum = pd.DataFrame({
            'Variable' : [], 'Quintile' : [], 'Coeff' : [], 't_val' : []
        })
        for col in range(6):
            Y = lead1wret_pivot.iloc[:, col]
            X = lead1wret_pivot.iloc[:, 6]
            reg_model = sm.OLS(Y, X).fit()
            coeff = round(reg_model.params[0], 4)
            t_val = round(reg_model.tvalues[0], 4)
            new_row = [var, col+1, coeff, t_val]
            db_reg_sum.loc[len(db_reg_sum.index)] = new_row
            
        db_reg_sum.to_csv(f"./output/RegSummaryTables/{var}_regsum_.csv", index=False)

        if verbose == 1:
            print(f"\nFinal table for {var} : \n {db_reg_sum} \n")
            print(f"\n Returns divided into qt for {var} : \n {qt_lead1wret.describe()} \n")
            print(f"\n Invitial returns : \n {db_var['lead1wret'].describe()} \n")
            if summary_pics == 1:
                summary_text = reg_model.summary().as_text()
                # Save summary text to pc
                fig, ax = plt.subplots(figsize=(8, 6))
                ax.axis('off')
                ax.text(
                    0.1, 
                    0.1, 
                    summary_text, 
                    va='top', 
                    ha='left', 
                    fontsize=10, 
                    family='monospace'
                )
                fig.savefig(
                    f'./output/RegSummariesPics/{var}/reg_res_{var}_{col+1}qt.png', 
                    bbox_inches='tight', 
                    pad_inches=0.5, dpi=300
                )

regression_tables(db_fin, var_list)
